In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os # Fürs Speichern

TARGET_URL = "https://www.sap.com/investors/de/financial-documents-and-events/recent-results.html"

# WICHTIG: Für SAP immer den ganzen Header aus Netwerkanalyse kopieren, User-Agent alleine geht nicht -> 403 Forbidden
HEADERS = {
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "de,ja;q=0.5",
    "Connection": "keep-alive",
    #"Cookie": "optout_domains_pc=; gpcishonored=true; notice_preferences=0:; notice_gdpr_prefs=0::implied,eu; notice_poptime=1726664400000; cmapi_gtm_bl=ga-ms-ua-ta-asp-bzi-sp-awct-cts-csm-img-flc-fls-mpm-mpr-m6d-tc-tdc; cmapi_cookie_privacy=permit 1 required; botinfo_sap=3819c28196071d507e51b1ca2ba9c8ff; country=DE; renderid=rend1; ngds_opt_out=true; TAsessionID=d5306e28-6143-4b67-86bc-ecca2cbb99aa|EXISTING; notice_behavior=implied,eu",
    #braucht nicht unbedingt Cookies
    "DNT": "1",
    "Host": "www.sap.com",
    "Priority": "u=0, i",
    "Referer": "https://www.sap.com/investors/de.html",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Sec-GPC": "1",
    "TE": "trailers",
    "Upgrade-Insecure-Requests": "1",
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:145.0) Gecko/20100101 Firefox/145.0'
}


# Speicherort
PATH_RAW_DATA = "../raw" # Pfad angepasst für Notebooks im Root
os.makedirs(PATH_RAW_DATA, exist_ok=True)

# Liste, in der wir die Daten (als Dictionaries) sammeln
collected_data = []


# --- 2. Phase: Download (Requests) ---
print(f"Lade HTML von: {TARGET_URL}")
try:
    response = requests.get(TARGET_URL, headers=HEADERS)
    # Prüfen, ob der Abruf erfolgreich war (Status 200 = OK)
    response.raise_for_status()
    print("Download erfolgreich.")

except requests.exceptions.RequestException as e:
    print(f"Fehler beim Abruf der URL: {e}")

Lade HTML von: https://www.sap.com/investors/de/financial-documents-and-events/recent-results.html
Download erfolgreich.


In [5]:

# --- 3. Phase: Parsing (BeautifulSoup) ---
# Erstelle ein "Suppen"-Objekt aus dem HTML-Text
soup = BeautifulSoup(response.text, 'html.parser') # "content" für Bilder usw. geeignet


# --- 4. Phase: Extraktion (Das ist Ihre Detektivarbeit!) ---
# Hier müssen Sie die Tags und Klassen einsetzen, die Sie in Phase 1 gefunden haben.
# BEISPIEL-STRUKTUR (wird bei SAP anders aussehen!)

# Suchen wir alle "News-Boxen" auf der Seite
# Angenommen, jede News ist in einem <article class="news-item">
news_items = soup.find_all('div', class_='Grid__row--SkLLf') # Beispiel!

if not news_items:
    print("WARNUNG: Keine 'news-item' gefunden. Prüfen Sie die HTML-Struktur!")
else:
    print(f"{len(news_items)} News-Items gefunden. Verarbeite...")

for item in news_items:
    try:
        # Finde die Überschrift (z.B. in einem <h2> mit class 'headline')
        headline_tag = item.find('h6', class_='Headline__root--lKpRX HeadlineWithLine__headline--YJJNM Headline__headline-xxxxs--gkvoY Headline__medium--irDyM Headline__color--A3sZu')
        headline = headline_tag.text.strip() if headline_tag else "N/A"

        # Finde das Datum (z.B. in einem <span> mit class 'date')
        date_tag = item.find('p', class_='Eyebrow__root--lBJPj TextStandard__eyebrow--Kcxze Eyebrow__eyebrow-s--DpVB7')
        date = date_tag.text.strip() if date_tag else "N/A"

        # Finde den Link zum vollen Artikel (im <a>-Tag)
        link_tag = item.find('a')
        link = link_tag['href'] if link_tag else "N/A"

        # HIER FOLGT DER "DEEP SCRAPE" (siehe unten)
        # Fürs Erste speichern wir nur die Metadaten

        collected_data.append({
            'company': 'SAP', # Hartkodiert, da wir nur SAP scrapen
            'date': date,
            'headline': headline,
            'url': link
            # 'full_text' kommt als Nächstes
        })

    except Exception as e:
        print(f"Fehler beim Parsen eines Items: {e}")

# --- 5. Phase: Speichern ---
df = pd.DataFrame(collected_data)
file_path = os.path.join(PATH_RAW_DATA, "sap_news_metadata.csv")
df.to_csv(file_path, index=False)

print(f"\nDaten erfolgreich in {file_path} gespeichert.")
print(df.head())

22 News-Items gefunden. Verarbeite...

Daten erfolgreich in ../raw/sap_news_metadata.csv gespeichert.
  company date             headline  \
0     SAP  N/A     22. Oktober 2025   
1     SAP  N/A                  N/A   
2     SAP  N/A      Auf einen Blick   
3     SAP  N/A                  N/A   
4     SAP  N/A  Dokumente und Links   

                                                 url  
0                                                N/A  
1                                                N/A  
2                                                N/A  
3                                                N/A  
4  /docs/download/investors/2025/sap-2025-q3-mitt...  
